# GRU Model Training

### Import Libraries, Download Data, Stock Data

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.optimizers import Adam

/Users/elizabethlarez/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
ticker = "AAPL"
data = yf.download(ticker, start="2010-01-01", end="2023-11-13")
df = data[['Close']].reset_index()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


### Preprocess Data and Create Sequences

In [3]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df['Close'].values.reshape(-1, 1))

def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:(i + seq_length), 0])
        y.append(data[i + seq_length, 0])
    return np.array(X), np.array(y)

seq_length = 60
X, y = create_sequences(scaled_data, seq_length)

### Split and Reshape Data

In [4]:
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

### Build and Train Model

In [5]:
model = Sequential([
    GRU(50, return_sequences=True, input_shape=(seq_length, 1)),
    GRU(50, return_sequences=False),
    Dense(25),
    Dense(1)
])
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.1, verbose=1)

Epoch 1/100


/Users/elizabethlarez/Library/Python/3.9/lib/python/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


78/78 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0051 - val_loss: 4.0022e-04
Epoch 2/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 1.4702e-05 - val_loss: 3.6638e-04
Epoch 3/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 1.5038e-05 - val_loss: 3.5091e-04
Epoch 4/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 1.5170e-05 - val_loss: 3.5071e-04
Epoch 5/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 1.3550e-05 - val_loss: 3.3645e-04
Epoch 6/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 1.5625e-05 - val_loss: 3.3780e-04
Epoch 7/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 1.3945e-05 - val_loss: 3.3593e-04
Epoch 8/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 1.4647e-05 - val_loss: 3.0017e-04
Epoch 9/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 1.2880e-05 - val_loss: 2.8796e-04
Epoch 10/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 1.3935e-05 - val_loss: 2.9988e-04
Epoch 11/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 1.2793e-05 - v

### Make Predictions and RMSE

In [6]:
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))
rmse = np.sqrt(np.mean((predictions - y_test)**2))
print(f"GRU RMSE: {rmse}")

22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
GRU RMSE: 2.8193985514879443
